In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

In [2]:
X,y = make_classification(n_samples=500, n_features=2, n_informative=2, n_redundant=0, n_repeated=0,
    n_classes=2, n_clusters_per_class=2, class_sep=0.6,flip_y=0.1, random_state=42)

In [3]:
df = pd.DataFrame(X,columns=['Feature 1','Feature 2'])
df['Target'] = y

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [6]:
y_train = y_train.reshape(400,1)
y_test = y_test.reshape(100,1)

In [7]:
class NeuralNetwork:
  def __init__(self,layers_dims):
    self.layers_dims = layers_dims

    self.weights = []
    self.biases = []

    for i in range(len(self.layers_dims)-1):
      self.weights.append(np.random.rand(self.layers_dims[i+1],self.layers_dims[i])*0.1)
      self.biases.append(np.zeros((1,self.layers_dims[i+1])))


  def sigmoid(self,Z):
    return 1/(1+np.exp(-Z))


  def sigmoid_derivative(self,Z):
    return Z*(1-Z)


  def forward_prop(self,X):
    activations = [X]

    for w,b in zip(self.weights,self.biases):
      Z = np.dot(activations[-1],w.T) + b
      A = self.sigmoid(Z)
      activations.append(A)

    return activations


  def bce(self,y_true,y_hat):
    m = y_true.shape[0]
    loss = -np.sum(y_true * np.log(y_hat) + (1 - y_true) * np.log(1 - y_hat)) / m
    return loss


  def bce_derivative(self,y_true,y_pred):
    return -(y_true / y_pred) + ((1 - y_true) / (1 - y_pred))



  def backward_prop(self,y_true,activations):
    dA = self.bce_derivative(y_true,activations[-1])                            # dL/dA
    m = y_true.shape[0]
    dWs = []
    dBs = []

    for i in reversed(range(len(self.weights))):
      dZ = self.sigmoid_derivative(activations[i+1]) * dA                         # dL/dZ = dL/dA * dA/dZ
      dw = np.dot(dZ.T,activations[i])/m
      db = np.sum(dZ,axis=0,keepdims=True)/m
      dA = np.dot(dZ,self.weights[i])

      dWs.insert(0,dw)
      dBs.insert(0,db)

    return dWs,dBs


  def update_coeff(self,dWs,dBs,lr):
    for i in range(len(self.weights)):
      self.weights[i] -= lr * dWs[i]
      self.biases[i] -= lr * dBs[i]


  def fit(self,X,y,epochs,learning_rate):
    for i in range(epochs):

      activations = self.forward_prop(X)

      loss = self.bce(y,activations[-1])

      dWs,dBs = self.backward_prop(y,activations)

      self.update_coeff(dWs,dBs,learning_rate)

      if i % 1000 == 0:
        print(f'epoch : {i} -> loss : {loss}')

  def predict(self,X):
    activation = self.forward_prop(X)
    return activation[-1]


In [18]:
nn = NeuralNetwork([2,2,1])

In [19]:
nn.fit(X_train,y_train,10000,0.5)

epoch : 0 -> loss : 0.6931085553679083
epoch : 1000 -> loss : 0.4772796529775286
epoch : 2000 -> loss : 0.4679016399202935
epoch : 3000 -> loss : 0.4648682379938376
epoch : 4000 -> loss : 0.4630979247586545
epoch : 5000 -> loss : 0.46186747587219373
epoch : 6000 -> loss : 0.46093590580328
epoch : 7000 -> loss : 0.4601916607118792
epoch : 8000 -> loss : 0.45957514334071203
epoch : 9000 -> loss : 0.4590516399883969


In [20]:
y_pred = nn.predict(X_test)

In [21]:
y_pred = np.where(y_pred>0.5,1,0)

In [22]:
from sklearn.metrics import accuracy_score
print('accuracy :', accuracy_score(y_test,y_pred))

accuracy : 0.84
